# ***Scandinavian News Articles***

This notebook is a part of the final project in ***Computational Tools for data science 02807*** and is written by *group 31* consisting of the members:

Felix Emil Bruun \
Eline Evje \
Malin Haugland Høli \
Ina Martini

Deadline: December 6, 2025

The main objective of the project is to analyze and compare political articles from various Scandinavian news outlets using machine learning, data mining techniques, and article classification.

## ***Imports***

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import re
from datasketch import MinHashLSH, MinHash
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_distances
from sentence_transformers import SentenceTransformer

## ***Import Dataset***

The datasets have been scraped using Selenium and BeautifulSoup.

In [ ]:
news_articles = pd.read_csv('data/articles.csv')

# information about the dataset
news_articles.shape[0]
news_articles.info()
news_articles.head(5)

## ***1. Preprocessing Steps***

### 1.1 Cleaning subscription boilerplate and layout noise

Many scraped news articles contain subscription prompts, “read more” links, and layout fragments that are not part of the actual content.

To avoid polluting the later similarity and clustering analyses, this has been removed from the `content` column. The resulting `content_clean` column is used for further text processing.

In [ ]:
# 1) Define "boilerplate" markers you want to cut away and everything after
SUBSCRIPTION_MARKERS = [
    "Allerede abonnent? Log ind",
    "Allerede abonnent? Log ind her",
    "Log ind for at læse",
    "Log ind for at læse hele artiklen",
    "Log ind for at læse videre",
    "Logg inn for å lese",
    "Bli abonnent",
    "Tegn abonnement",
    "Få adgang til hele artiklen",
    "Få adgang til alt indhold",

    # Swedish / Norwegian / Danish "read more" prompts
    "Läs mer",       # SE
    "Læs mere",      # DK
    "Les mer",       # NO

    # DN print-view header
    "En utskrift från Dagens Nyheter",
]

def clean_content(text: str) -> str:
    if pd.isna(text):
        return ""
    t = str(text)

    # --- basic formatting fixes ---
    # remove leading "/ " like "/ Under 30 minuter ..."
    t = re.sub(r'^\s*/\s*', '', t)

    # remove bullet characters that often appear in scraped content
    t = t.replace("•", " ")

    # --- cut off subscription / boilerplate tail ---
    lower = t.lower()
    cut_pos = len(t)
    for marker in SUBSCRIPTION_MARKERS:
        idx = lower.find(marker.lower())
        if idx != -1 and idx < cut_pos:
            cut_pos = idx

    t = t[:cut_pos]

    # --- normalize whitespace ---
    t = re.sub(r'\s+', ' ', t).strip()

    return t

# Apply to your dataframe
news_articles['content_clean'] = news_articles['content'].apply(clean_content)

news_articles.head(5)

### 1.2 Remove irrelevant links

In some norwegian articles posted by "vg.no" there are links to another website called "e24.no". These are not relevant for the project, and are therefore dropped from the data frame

In [ ]:
# drop all norwegian articles from "vg" that contains "e24" in the url
e24_links = news_articles[(news_articles['country'] == 'norway') & (news_articles['source'] == 'vg') & (news_articles['url'].str.contains('https://e24.no'))]
news_articles = news_articles.drop(e24_links.index)
news_articles.reset_index(drop=True, inplace=True)

# print number of articles after preprocessing
print("Number of articles after preprocessing:", news_articles.shape[0])

### 1.3 Remove invalid source

During scraping the source "news" has been included, which is not a valid source. Rows with this source has also been dropped. 

In [ ]:
# count rows with source "news"
invalid_source = news_articles[news_articles['source'] == 'news']
print("Number of articles with source 'news':", invalid_source.shape[0])

# drop rows with source "news"
news_articles = news_articles[news_articles['source'] != 'news']
news_articles.reset_index(drop=True, inplace=True)
print("Number of articles after dropping 'news' source:", news_articles.shape[0])

### 1.4 Standardize source names

Some sources have uppercase letters and some not, so we standardize by only using lowercase.

In [ ]:
news_articles['source'] = news_articles['source'].str.lower()

### 1.5 Remove articles outside the intended time period 2020-2025

In [ ]:
news_articles['date'] = pd.to_datetime(news_articles['date'], errors='coerce')
news_articles = news_articles[(news_articles['date'] >= '2020-01-01') & (news_articles['date'] <= '2025-12-31')]
news_articles.reset_index(drop=True, inplace=True)
print("Number of articles after removing those outside 2020-2025:", news_articles.shape[0])

### 1.6 Remove duplicates

In [ ]:
# 1. Drop strict duplicates (same URL + same content + same headline)
news_articles = news_articles.drop_duplicates(
    subset=['url', 'source', 'content_clean', 'headline']
).reset_index(drop=True)

print("After strict duplicates:", news_articles.shape[0])

# 2. Drop soft duplicates (same content/headline but different URL)
news_articles = news_articles.drop_duplicates(
    subset=['content_clean', 'headline']
).reset_index(drop=True)

print("After soft duplicates:", news_articles.shape[0])

### 1.7 Drop articles with worldview score 6, i.e., not classified

In [ ]:
news_articles = news_articles[news_articles['worldview_score'] != 6]
news_articles.reset_index(drop=True, inplace=True)
print("Number of articles after dropping unclassified worldview score:", news_articles.shape[0])

## ***2. Dataset Exploration***

### 2.1 Source information

The table below presents information about the *sources* (news outlets) that has been retrieved. There are 6 sources from Sweden, 5 from Denmark and 4 from Norway. The dataset for the analysis includes articles from 2020-2025. 

In [ ]:
unique_sources = news_articles[['country', 'source']].drop_duplicates().sort_values(by='country')

sources_per_country = unique_sources['country'].value_counts()
display(HTML("<h4 style='font-weight:700'>Number of Sources per Country:</h4>"))
display(HTML("<pre style='font-family:monospace'>{}</pre>".format(
    sources_per_country.rename(index=lambda s: s.capitalize()).to_string(header=False)
)))

src_counts = (news_articles
              .groupby(['country', 'source'])
              .size()
              .reset_index(name='n_articles'))

# compute average worldview_score per country and source
temp_view = news_articles.copy()
temp_view['worldview_num'] = pd.to_numeric(temp_view['worldview_score'], errors='coerce')
avg_world = temp_view.groupby(['country', 'source'])['worldview_num'].mean().reset_index(name='avg_worldview').round(2)
display_df = src_counts.merge(avg_world, on=['country', 'source'], how='left')

# add column showing the time range of articles per country and source
time_ranges = (news_articles
                .groupby(['country', 'source'])
                .agg(first_article=('date', 'min'), last_article=('date', 'max'))
                .reset_index())

time_ranges['time_range'] = (
    time_ranges['first_article'].dt.strftime('%Y-%m-%d') +
    ' to ' +
    time_ranges['last_article'].dt.strftime('%Y-%m-%d')
)

display_df = display_df.merge(time_ranges[['country', 'source', 'time_range']], on=['country', 'source'])  
display_df = display_df[['source', 'n_articles', 'avg_worldview', 'country', 'time_range']]
display_df = display_df.sort_values(by='country').copy()
display(HTML(display_df.to_html(index=False, border=0)))

### 2.2 Article distribution per year in the different countries

In [ ]:
countries = news_articles['country'].unique()
num_countries = len(countries)

fig, axes = plt.subplots(1, num_countries, figsize=(18, 6), sharey=True)

if num_countries == 1:
    axes = [axes]

for ax, country in zip(axes, countries):
    country_data = news_articles[news_articles['country'] == country].copy()

    for i, source in enumerate(country_data['source'].unique()):
        source_data = country_data[country_data['source'] == source].copy()
        source_data['year'] = source_data['date'].dt.year
        year_counts = source_data['year'].value_counts().sort_index()

        ax.bar(
            year_counts.index + (0.1 * i),
            year_counts.values,
            width=0.1,
            label=source
        )

    ax.set_title(f'Articles per Year – {country.capitalize()}')
    ax.set_xlabel('Year')
    plt.figtext(0.5, -0.01, 'Figure 1: Shows the distribution of articles by year and source. Norway has the least amount of articles.', ha='center', fontsize=10)
    ax.set_xticks(range(2020, 2026))
    ax.legend()

axes[0].set_ylabel('Number of Articles')

plt.tight_layout()
plt.show()

### 2.3 Average worldview in each country from 2020-2025

In [ ]:
import matplotlib.pyplot as plt
worldview_over_time = (news_articles
                       .copy())
worldview_over_time['worldview_num'] = pd.to_numeric(worldview_over_time['worldview_score'], errors='coerce')
worldview_over_time['year'] = worldview_over_time['date'].dt.year
avg_worldview_time = (worldview_over_time
                      .groupby(['country', 'year'])['worldview_num']
                      .mean()
                      .reset_index())

plt.figure(figsize=(10, 6))
for country in avg_worldview_time['country'].unique():
    country_data = avg_worldview_time[avg_worldview_time['country'] == country]
    plt.plot(country_data['year'], country_data['worldview_num'], marker='o', label=country.capitalize())

    
plt.title('Average Worldview Score Over Time by Country')
plt.xlabel('Year')
plt.ylabel('Average Worldview Score')
plt.figtext(0.5, 0.01, 'Figure 2: Illustrates the average worldview score over time by country based on the news articles.', ha='center', fontsize=10)
plt.xticks(avg_worldview_time['year'].unique())
plt.legend()
plt.grid()
plt.show()

## ***3. Data Analysis***

### 3.1 Implementing multilingual embeddings

Before comparing similarity methods (LSH, ANN, and clustering), a transformers model was used to map the different languages to a shared space. The result is is a embedding matrix that will be used for further analysis.

In [ ]:
# Use headline + cleaned content for richer semantics
texts = (
    news_articles['headline'].fillna('') + ' ' +
    news_articles['content_clean'].fillna('')
).astype(str).tolist()

# Multilingual model that maps different languages to a shared space
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(model_name)

# Dense embedding matrix: shape (n_docs, embedding_dim)
emb_matrix = embed_model.encode(
    texts,
    convert_to_numpy=True,
    show_progress_bar=True,
    batch_size=64  # tweak if GPU/CPU memory is small
)

print("Embedding matrix shape:", emb_matrix.shape)

### 3.2 Approximate Nearest Neighbor (ANN)

A ANN uses a NearestNeighbors index over the TF-IDF vectors to quickly retrieve the articles most similar to a given one. 

In [ ]:
def build_ann_index(matrix, n_neighbors=10, metric='cosine'):
    """
    Build an ANN index using scikit-learn's NearestNeighbors.
    Works with TF-IDF or any sparse matrix.
    """
    nn = NearestNeighbors(
        n_neighbors=n_neighbors + 1,  
        metric=metric,
        algorithm='brute'  # sparse matrices
    )
    nn.fit(matrix)
    return nn

# Build ANN index on embeddings
ann_index = build_ann_index(emb_matrix, n_neighbors=10)

def find_similar_articles(query_idx, ann_index, matrix, n_neighbors=10):
    """
    Find most similar articles to a given article using ANN.
    Returns indices and distances of similar articles.
    """
    query_vector = matrix[query_idx:query_idx+1]
    distances, indices = ann_index.kneighbors(query_vector)
    
    # Remove the query article itself (first result)
    return indices[0][1:], distances[0][1:]

In [ ]:
def print_similar_articles(query_idx, ann_index, matrix, news_df, n):
    # Find ANN neighbors
    indices, distances = find_similar_articles(query_idx, ann_index, matrix, n_neighbors=n)

    # Build table
    rows = []
    news_df.loc[query_idx, 'country']

    for idx, dist in zip(indices, distances):
        rows.append({
            'similarity': round(1 - dist, 3),
            'country': news_df.loc[idx, 'country'],
            'source': news_df.loc[idx, 'source'],
            'headline': news_df.loc[idx, 'headline'],
            'clean_content': news_df.loc[idx, 'content_clean'],
            'worldview_score': news_df.loc[idx, 'worldview_score']
        })

    df = pd.DataFrame(rows)

    # Example using article 5 as query
    display(HTML(f"<h4 style='font-weight:700'>Example article {query_idx}:</h4>"))
    print(f"Headline: {news_df.loc[query_idx, 'headline']}")
    print(f"Country: {news_df.loc[query_idx, 'country']} | Source: {news_df.loc[query_idx, 'source']}")
    print(f"Worldview Score: {news_df.loc[query_idx, 'worldview_score']}\n")

    display(HTML("<h4 style='font-weight:700'>Most similar articles:</h4>"))
    display(df.head(n))

print_similar_articles(280, ann_index, emb_matrix, news_articles, n=10)

#### ANN Analysis by Country and Worldview

This analysis checks how often each article’s most semantically similar neighbors come from the same country or share the same worldview score.


In [ ]:
def analyze_similarities_by_country(ann_index, matrix, news_df, n_neighbors=10):
    """
    Analyze how similar articles cluster by country and worldview.
    """
    country_similarities = []
    worldview_similarities = []
    
    for i in range(len(news_df)):
        similar_indices, distances = find_similar_articles(i, ann_index, matrix, n_neighbors)
        
        query_country = news_df.loc[i, 'country']
        query_worldview = news_df.loc[i, 'worldview_score']
        
        # Count same-country neighbors
        same_country = sum(1 for idx in similar_indices 
                          if news_df.loc[idx, 'country'] == query_country)
        country_similarities.append(same_country / len(similar_indices))
        
        # Count same-worldview neighbors
        same_worldview = sum(1 for idx in similar_indices 
                           if news_df.loc[idx, 'worldview_score'] == query_worldview)
        worldview_similarities.append(same_worldview / len(similar_indices))
    
    return country_similarities, worldview_similarities

country_sim, worldview_sim = analyze_similarities_by_country(ann_index, emb_matrix, news_articles)

# Breakdown by country
country_breakdown = news_articles.copy()
country_breakdown['country_sim_ratio'] = country_sim
country_breakdown['worldview_sim_ratio'] = worldview_sim

country_stats = (country_breakdown
                .groupby('country')
                .agg({
                    'country_sim_ratio': 'mean',
                    'worldview_sim_ratio': 'mean'
                })
                .round(3))

country_stats_plot = country_stats.reset_index()

fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=False)

# --- Left bar plot: Same-country ratio ---
axes[0].bar(country_stats_plot['country'], country_stats_plot['country_sim_ratio'])
axes[0].set_title("Same-Country Neighbor Ratio")
axes[0].set_xlabel("Country")
axes[0].set_ylabel("Ratio")

# --- Right bar plot: Same-worldview ratio ---
axes[1].bar(country_stats_plot['country'], country_stats_plot['worldview_sim_ratio'])
axes[1].set_title("Same-Worldview Neighbor Ratio")
axes[1].set_xlabel("Country")

plt.figtext(0.5, -0.01, 'Figure 3: Compares the average ratios of same-country and same-worldview neighbors found via ANN similarity search across the different countries.', ha='center', fontsize=10)
plt.tight_layout(rect=[0, 0.03, 1, 1])
plt.show()

display(HTML("<h4>Similarity Analysis by Country</h4>"))
display(HTML(country_stats.to_html()))

#### Measure ideological polarization within each media outlet
The polarization score reflects how semantically similar articles within a news outlet tend to share the same worldview rating, where higher values indicate stronger ideological alignment and lower values suggest greater ideological diversity within the outlet’s content.

In [ ]:
def outlet_polarization(df, ann_index, matrix, n=10):
    scores = []
    for i in range(len(df)):
        sim_idx, _ = find_similar_articles(i, ann_index, matrix, n)
        target = df.loc[i, 'worldview_score']
        same = sum(df.loc[idx, 'worldview_score'] == target for idx in sim_idx)
        scores.append(same / len(sim_idx))
    df['polarization'] = scores

    # sort by values (highest polarization first)
    return df.groupby('source')['polarization'].mean().round(3).sort_values(ascending=True)

outlet_polarization(news_articles, ann_index, tfidf_matrix)


#### Semantic Ideological Consistency
The polarization score shows how semantically similar articles within a country tend to share the same worldview rating, where higher values indicate stronger ideological alignment.


In [ ]:
def country_polarization(df, matrix, ann_index, n=10):
    scores = []
    for i in range(len(df)):
        sim_idx, _ = find_similar_articles(i, ann_index, matrix, n)
        target = df.loc[i, 'worldview_score']
        same = sum(df.loc[idx, 'worldview_score'] == target for idx in sim_idx)
        scores.append(same / len(sim_idx))

    df['polarization'] = scores
    return df.groupby('country')['polarization'].mean().sort_values().round(3)

country_polarization(news_articles, tfidf_matrix, ann_index)


### 3.3 DBSCAN Clustering

Clustering can detect *groups of similar news articles* that emerge naturally from their content.

By clustering the articles and then inspecting the cluster contents, it is possible to e.g explore whether articles with similar ***worldview score*** appear in the same clusters. 

In [ ]:
docs = news_articles['content_clean'].astype(str).tolist()

vect = CountVectorizer(analyzer='char', ngram_range=(5,5), binary=True)
X = vect.fit_transform(docs)

# Compute pairwise Jaccard distances (1 - similarity)
distances = pdist(X.toarray(), metric="jaccard")

# Convert to a matrix
dist_matrix = squareform(distances)

# Convert to similarity:
sim_matrix = 1 - dist_matrix

db = DBSCAN(
    eps=0.905,     
    min_samples=7,
    metric='precomputed'
)

labels = db.fit_predict(dist_matrix)

cluster_counts = pd.Series(labels).value_counts().sort_index()
print(cluster_counts)

In [ ]:
def perform_dbscan_clustering(matrix, eps=0.3, min_samples=5, metric='cosine'):
    """
    Perform DBSCAN clustering on the feature matrix.
    For cosine metric, we compute distances and use precomputed.
    """
    if metric == 'cosine':
        # Convert sparse matrix to dense for cosine distance calculation
        dense_matrix = matrix.toarray() if hasattr(matrix, 'toarray') else matrix
        distance_matrix = cosine_distances(dense_matrix)
        
        dbscan = DBSCAN(
            eps=eps,
            min_samples=min_samples,
            metric='precomputed'
        )
        labels = dbscan.fit_predict(distance_matrix)
    else:
        dbscan = DBSCAN(
            eps=eps,
            min_samples=min_samples,
            metric=metric
        )
        labels = dbscan.fit_predict(matrix.toarray() if hasattr(matrix, 'toarray') else matrix)
    
    return labels

def analyze_clusters(labels, news_df):
    """
    Analyze the clustering results.
    """
    cluster_df = news_df.copy()
    cluster_df['cluster'] = labels
    
    # Count clusters
    cluster_counts = pd.Series(labels).value_counts().sort_index()
    n_clusters = len(cluster_counts[cluster_counts.index != -1])
    n_noise = cluster_counts.get(-1, 0)
    
    print(f"Number of clusters: {n_clusters}")
    print(f"Number of noise points: {n_noise}")
    print(f"Clustering ratio: {(len(labels) - n_noise) / len(labels):.3f}")
    
    return cluster_df, cluster_counts

# Test different eps values
eps_values = [0.2, 0.3, 0.4, 0.5]
clustering_results = {}

for eps in eps_values:
    labels = perform_dbscan_clustering(tfidf_matrix, eps=eps, min_samples=3)
    cluster_df, cluster_counts = analyze_clusters(labels, news_articles)
    clustering_results[eps] = (labels, cluster_df, cluster_counts)
    
    print(f"\nEps={eps}:")
    n_clusters = len(cluster_counts[cluster_counts.index != -1])
    n_noise = cluster_counts.get(-1, 0)
    print(f"  Clusters: {n_clusters}, Noise: {n_noise}")

In [ ]:
def visualize_clustering_results(clustering_results):
    """
    Visualize DBSCAN clustering results for different eps values.
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, (eps, (labels, cluster_df, cluster_counts)) in enumerate(clustering_results.items()):
        ax = axes[i]
        
        # Plot cluster sizes (excluding noise)
        valid_clusters = cluster_counts[cluster_counts.index != -1]
        
        if len(valid_clusters) > 0:
            ax.bar(range(len(valid_clusters)), valid_clusters.values)
            ax.set_title(f'Eps={eps}: {len(valid_clusters)} clusters, {cluster_counts.get(-1, 0)} noise')
            ax.set_xlabel('Cluster ID')
            ax.set_ylabel('Number of Articles')
        else:
            ax.text(0.5, 0.5, f'No clusters found\n(eps={eps})', 
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'Eps={eps}: No clusters')
    
    plt.tight_layout()
    plt.figtext(0.5, 0.01, 'Figure 3: DBSCAN clustering results for different eps values.', ha='center', fontsize=10)
    plt.show()

visualize_clustering_results(clustering_results)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def summarize_clustering_results(clustering_results):
    """
    Summarize DBSCAN results across eps values:
    - number of clusters (excluding noise)
    - fraction of points labeled as noise
    """
    rows = []
    for eps, (labels, cluster_df, cluster_counts) in clustering_results.items():
        total = len(labels)
        n_noise = cluster_counts.get(-1, 0)
        n_clusters = (cluster_counts.index != -1).sum()
        noise_ratio = n_noise / total if total > 0 else 0.0
        rows.append((eps, n_clusters, noise_ratio))

    rows = sorted(rows, key=lambda x: x[0])
    eps_values = [r[0] for r in rows]
    n_clusters = [r[1] for r in rows]
    noise_ratios = [r[2] for r in rows]

    fig, ax1 = plt.subplots(figsize=(8, 5))

    # Bar: number of clusters
    x = np.arange(len(eps_values))
    ax1.bar(x, n_clusters)
    ax1.set_xlabel("eps")
    ax1.set_ylabel("Number of clusters")
    ax1.set_xticks(x)
    ax1.set_xticklabels(eps_values)

    # Line: fraction of noise points
    ax2 = ax1.twinx()
    ax2.plot(x, noise_ratios, marker='o')
    ax2.set_ylabel("Fraction of noise points")

    plt.title("DBSCAN behavior across eps values")
    plt.tight_layout()
    plt.show()

# usage
summarize_clustering_results(clustering_results)


In [ ]:
import umap
import matplotlib.pyplot as plt
import numpy as np

def visualize_clusters_2d(emb_matrix, labels, title="Cluster Visualization (2D UMAP)"):
    """
    Project embedding matrix to 2D and color points by cluster label.
    Noise points (-1) get their own color (usually black/gray).
    """
    reducer = umap.UMAP(random_state=42)
    emb_2d = reducer.fit_transform(emb_matrix)

    unique_labels = np.unique(labels)
    colors = plt.cm.tab20(np.linspace(0, 1, len(unique_labels)))

    plt.figure(figsize=(10, 7))

    for lab, col in zip(unique_labels, colors):
        mask = labels == lab
        if lab == -1:
            # Noise points
            plt.scatter(
                emb_2d[mask, 0], emb_2d[mask, 1],
                c="lightgrey", s=12, label="Noise (-1)", alpha=0.6
            )
        else:
            plt.scatter(
                emb_2d[mask, 0], emb_2d[mask, 1],
                c=[col], s=20, label=f"Cluster {lab}"
            )

    plt.title(title)
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.legend(loc="best", markerscale=1.5, fontsize=8)
    plt.tight_layout()
    plt.show()


In [ ]:
# Example: visualize UMAP clusters for eps = 0.9
eps = 0.9
labels, cluster_df, cluster_counts = clustering_results[eps]

visualize_clusters_2d(emb_matrix, labels, title=f"DBSCAN Clusters (eps={eps})")


#### Examine cluster content

To interpret the DBSCAN clusters, the function below prints:

- The number of articles in the cluster.
- The distribution of countries, sources and worldview scores.
- A small sample of article headlines and the first part of their content. 

In [ ]:
def examine_cluster_content(cluster_df, cluster_id, max_articles=5):
    """
    Examine the content of a specific cluster.
    """
    cluster_articles = cluster_df[cluster_df['cluster'] == cluster_id]
    
    print(f"\nCluster {cluster_id} ({len(cluster_articles)} articles):")
    print("Country distribution:", cluster_articles['country'].value_counts().to_dict())
    print("Source distribution:", cluster_articles['source'].value_counts().to_dict())
    print("Worldview distribution:", cluster_articles['worldview_score'].value_counts().to_dict())
    
    print(f"\nSample articles (showing up to {max_articles}):")
    for i, (idx, row) in enumerate(cluster_articles.head(max_articles).iterrows()):
        print(f"{i+1}. [{row['source']}] {row['headline']}")
        print(f"   Content: {row['content_clean'][:100]}...\n")

# Choose best eps value and examine clusters
best_eps = 0.3  # adjust based on your results
_, best_cluster_df, best_cluster_counts = clustering_results[best_eps]

# Show largest clusters
valid_clusters = best_cluster_counts[best_cluster_counts.index != -1]
largest_clusters = valid_clusters.nlargest(3)

for cluster_id in largest_clusters.index:
    examine_cluster_content(best_cluster_df, cluster_id)

### 3.4 Locality Sensitive Hashing

The LSH implementation below builds a *simple MinHash + LSH pipeline* as a baseline for detecting near-duplicate articles.

The main steps are:

1. MinHash signatures  
   Each article is represented by a MinHash object built from word shingles.  
   These signatures approximate the Jaccard similarity between articles.

2. MinHashLSH index 
   Each MinHash is inserted into a `MinHashLSH` index with a chosen similarity threshold.  
   This makes it possible to query similar articles without having to check all pairs. 

3. Enumerating similar pairs
   The implementation loops over all articles, queries the LSH index for neighbors, and collects all unique pairs of articles that LSH thinks are similar.

The headline and content is combined in order to capture the most meaning of an article. 

In [ ]:
news_articles['combined'] = news_articles['headline'].fillna('') + ' ' + news_articles['content_clean'].fillna('')

def get_shingles(text, k=5):
    """
    Create word shingles (k-grams) from a text.
    Example: k=5 means sequences of 5 consecutive words.
    """
    # basic tokenization: keep alphanumeric words
    tokens = re.findall(r'\w+', str(text).lower())
    if len(tokens) < k:
        return set([' '.join(tokens)]) if tokens else set()
    return {
        ' '.join(tokens[i:i + k])
        for i in range(len(tokens) - k + 1)
    }

# Quick test on one article:
example_shingles = get_shingles(news_articles['combined'].iloc[3], k=5)
print("Number of shingles:", len(example_shingles)) 
print("The first 5 shingles:", list(example_shingles)[:5])

In [ ]:
def create_minhash(shingles, num_perm=128):
    """
    Build a MinHash signature from a set of shingles.
    num_perm controls the signature size (and accuracy).
    """
    m = MinHash(num_perm=num_perm)
    for sh in shingles:
        m.update(sh.encode('utf-8'))
    return m

num_perm = 128
k = 5  # shingle size

minhashes = {}

for idx, text in news_articles['text'].items():
    shingles = get_shingles(text, k=k)
    m = create_minhash(shingles, num_perm=num_perm)
    minhashes[idx] = m

print("Created MinHash signatures for", len(minhashes), "articles.")

In [ ]:
class EnhancedLSH:
    """
    LSH implementation with multiple hash families and analysis tools.
    """
    
    def __init__(self, threshold=0.5, num_perm=128, num_bands=None):
        self.threshold = threshold
        self.num_perm = num_perm
        self.num_bands = num_bands or max(1, num_perm // 8)
        self.lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
        self.minhashes = {}
        self.metadata = {}
        
    def add_document(self, doc_id, text, metadata=None):
        """
        Add a document to the LSH index.
        """
        shingles = get_shingles(text, k=5)
        minhash = create_minhash(shingles, self.num_perm)
        
        self.lsh.insert(str(doc_id), minhash)
        self.minhashes[doc_id] = minhash
        
        if metadata:
            self.metadata[doc_id] = metadata
    
    def query_similar(self, doc_id, return_scores=True):
        """
        Find similar documents to a given document.
        """
        if doc_id not in self.minhashes:
            return []
            
        query_hash = self.minhashes[doc_id]
        candidates = self.lsh.query(query_hash)
        
        results = []
        for candidate in candidates:
            candidate_id = int(candidate)
            if candidate_id == doc_id:
                continue
                
            if return_scores:
                similarity = query_hash.jaccard(self.minhashes[candidate_id])
                results.append((candidate_id, similarity))
            else:
                results.append(candidate_id)
        
        if return_scores:
            results.sort(key=lambda x: x[1], reverse=True)
            
        return results
    
    def get_all_similar_pairs(self):
        """
        Get all similar pairs found by LSH.
        """
        similar_pairs = set()
        
        for doc_id in self.minhashes:
            neighbors = self.query_similar(doc_id, return_scores=False)
            for neighbor_id in neighbors:
                pair = tuple(sorted((doc_id, neighbor_id)))
                similar_pairs.add(pair)
        
        return similar_pairs

# LSH index
enhanced_lsh = EnhancedLSH(threshold=0.4, num_perm=128)

for idx, row in news_articles.iterrows():
    metadata = {
        'country': row['country'],
        'source': row['source'],
        'worldview_score': row['worldview_score'],
        'headline': row['headline']
    }
    enhanced_lsh.add_document(idx, row['text'], metadata)

print(f"LSH index built with {len(enhanced_lsh.minhashes)} documents")

#### LSH Performance

Given an `EnhancedLSH` instance, it:

1. Retrieves all similar article pairs that LSH has found.
2. Computes the true MinHash Jaccard similarity for each pair.
3. Tracks how often similar pairs share:
   - the same ***country***,
   - the same ***source***,
   - the same ***worldview score***.

In [ ]:
def analyze_lsh_performance(enhanced_lsh, news_df):
    """
    Analyze LSH performance and similarity patterns.
    """
    all_pairs = enhanced_lsh.get_all_similar_pairs()
    
    print(f"Total similar pairs found: {len(all_pairs)}")
    
    # Analyze similarity distribution
    similarities = []
    country_matches = 0
    source_matches = 0
    worldview_matches = 0
    
    for doc1, doc2 in all_pairs:
        sim = enhanced_lsh.minhashes[doc1].jaccard(enhanced_lsh.minhashes[doc2])
        similarities.append(sim)
        
        meta1 = enhanced_lsh.metadata[doc1]
        meta2 = enhanced_lsh.metadata[doc2]
        
        if meta1['country'] == meta2['country']:
            country_matches += 1
        if meta1['source'] == meta2['source']:
            source_matches += 1
        if meta1['worldview_score'] == meta2['worldview_score']:
            worldview_matches += 1
    
    if len(all_pairs) > 0:
        print(f"Average similarity: {np.mean(similarities):.3f}")
        print(f"Same country ratio: {country_matches / len(all_pairs):.3f}")
        print(f"Same source ratio: {source_matches / len(all_pairs):.3f}")
        print(f"Same worldview ratio: {worldview_matches / len(all_pairs):.3f}")
    
    return similarities, all_pairs

similarities, similar_pairs = analyze_lsh_performance(enhanced_lsh, news_articles)

# Show top similar pairs
pair_details = []
for doc1, doc2 in list(similar_pairs)[:10]:
    sim = enhanced_lsh.minhashes[doc1].jaccard(enhanced_lsh.minhashes[doc2])
    pair_details.append((sim, doc1, doc2))

pair_details.sort(reverse=True)

print("\nTop 3 most similar pairs:")
for sim, doc1, doc2 in pair_details[:3]:
    print(f"\nSimilarity: {sim:.3f} | Pair: {doc1} – {doc2}")
    print(f"A: [{news_articles.loc[doc1, 'source']}] {news_articles.loc[doc1, 'headline']}"
          f"\nContent: {news_articles.loc[doc1, 'content_clean'][:150]}")
    print(f"B: [{news_articles.loc[doc2, 'source']}] {news_articles.loc[doc2, 'headline']}"
        f"\nContent: {news_articles.loc[doc2, 'content_clean'][:150]}") 

To compare different LSH similarity thresholds, the LSH pipeline runs multiple times with varying `threshold` values.

The table reveals what threshold that balances: 
- *coverage* (not miss interesting similar articles),
- and *precision* (avoid flooding the analysis with weak matches).


In [ ]:
def compare_lsh_thresholds(thresholds, news_df):
    """
    Compare LSH performance across different similarity thresholds.
    """
    results_summary = []
    
    for threshold in thresholds:
        lsh_test = EnhancedLSH(threshold=threshold, num_perm=128)
        
        for idx, row in news_df.iterrows():
            metadata = {
                'country': row['country'],
                'source': row['source'],
                'worldview_score': row['worldview_score']
            }
            lsh_test.add_document(idx, row['text'], metadata)
        
        pairs = lsh_test.get_all_similar_pairs()
        
        if len(pairs) > 0:
            sims, _ = analyze_lsh_performance(lsh_test, news_df)
            avg_sim = np.mean(sims)
        else:
            avg_sim = 0
        
        results_summary.append({
            'threshold': threshold,
            'n_pairs': len(pairs),
            'avg_similarity': avg_sim
        })
            
    return pd.DataFrame(results_summary)

# Test different thresholds
thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
threshold_results = compare_lsh_thresholds(thresholds, news_articles)

display(HTML("<h4>LSH Threshold Comparison</h4>"))
display(HTML(threshold_results.to_html(index=False)))

### LSH vs ANN Comparison

In [ ]:
def compare_ann_vs_lsh(query_idx, ann_index, enhanced_lsh, matrix, news_df, n_neighbors=10):
    """
    Compare ANN and LSH results for a specific query article.
    """
    # ANN results
    ann_indices, ann_distances = find_similar_articles(query_idx, ann_index, matrix, n_neighbors)
    ann_similarities = 1 - ann_distances
    
    # LSH results
    lsh_results = enhanced_lsh.query_similar(query_idx, return_scores=True)
    lsh_indices = [idx for idx, _ in lsh_results]
    lsh_similarities = [sim for _, sim in lsh_results]
    
    print(f"Query article {query_idx}: '{news_df.loc[query_idx, 'headline']}'")
    print(f"Country: {news_df.loc[query_idx, 'country']} | Source: {news_df.loc[query_idx, 'source']}\n")
    
    print(f"ANN found {len(ann_indices)} neighbors")
    print(f"LSH found {len(lsh_indices)} neighbors\n")
    
    # Show overlap
    ann_set = set(ann_indices)
    lsh_set = set(lsh_indices)
    overlap = ann_set.intersection(lsh_set)
    
    print(f"Overlap: {len(overlap)} articles")
    print(f"ANN-only: {len(ann_set - lsh_set)} articles")
    print(f"LSH-only: {len(lsh_set - ann_set)} articles\n")
    
    # top 3 results
    print("Top 3 ANN results:")
    for i, (idx, sim) in enumerate(zip(ann_indices[:3], ann_similarities[:3])):
        print(f"  {i+1}. Sim: {sim:.3f} | [{news_df.loc[idx, 'source']}] {news_df.loc[idx, 'headline']}")
    
    print("\nTop 3 LSH results:")
    for i, (idx, sim) in enumerate(lsh_results[:3]):
        print(f"  {i+1}. Sim: {sim:.3f} | [{news_df.loc[idx, 'source']}] {news_df.loc[idx, 'headline']}")

    if len(overlap) == 0:
        print("\nNo overlapping articles by LSH.")

    if len(overlap) > 0:
        print("\nOverlapping articles:")
        for idx in overlap:
            print(f"  [{news_df.loc[idx, 'source']}] {news_df.loc[idx, 'headline']}")

# compare with sampled article
sample_article = 50

print("=" * 80)
compare_ann_vs_lsh(sample_article, ann_index, enhanced_lsh, tfidf_matrix, news_articles, n_neighbors=5)
print()